In [3]:
__author__ = "me"
__date__ = "2015_10_13"

%pylab inline
import pandas as pd
import numpy as np
import geopandas as gp

import pylab as plt
import os

import requests
s = requests.get("https://raw.githubusercontent.com/Casyfill/CUSP_templates/master/Py/fbMatplotlibrc.json").json()
plt.rcParams.update(s)

PARQA = os.getenv('PARQA')

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [4]:
litter = pd.read_csv(PARQA + "data/All_Inspections.csv",index_col=0)
litter = litter[litter.Litter==1]
litter.Date = pd.to_datetime(litter.Date, format='%Y-%m-%d')

In [ ]:
litterCallDistr = pd

In [ ]:
def getBefore(x)